In [1]:
model_name = "distilroberta-finetuned-financial-news-sentiment-analysis_fr_gr"

In [2]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nestorojedagonzalez (nojeda) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'data/train_subset.csv', 'eval': 'data/eval_subset.csv'})

In [4]:
from utils import transform_labels
dataset = dataset.map(transform_labels)

In [5]:
# let's train a Distilbert model

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')

# let's tokenize the data for the model to be able to understand
def tokenize_data(example):
    return tokenizer(example['Sentence'], padding='max_length')    

In [6]:
dataset = dataset.map(tokenize_data, batched=True)

In [7]:
from transformers import AutoModelForSequenceClassification

# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", num_labels=3)

In [8]:
# the default batch size for training arguments
batch_size = 8

# set number of epochs
number_of_epochs = 7
# let set the logging steps
logging_steps = len(dataset['train']) // batch_size # it should log each batch 

steps = (len(dataset['train']) / batch_size) * number_of_epochs
warmup_steps = int(0.2 * steps)

In [9]:
from transformers import TrainingArguments
training_args = TrainingArguments(
                                  num_train_epochs=number_of_epochs, 
                                  load_best_model_at_end=True,
                                  evaluation_strategy='steps', 
                                  save_strategy='steps',
                                  learning_rate=2e-5,
                                  logging_steps=logging_steps,
                                  warmup_steps= warmup_steps,
                                  save_steps=1000,
                                  eval_steps=500,
                                  output_dir=model_name,
                                  report_to='wandb',
                                  )

e:\repo\DistilBERTFinancialSentiment\.venv\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['eval'].shuffle(seed=10)

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [12]:
# Launch the learning process: training
# Load model from checkpoint
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
500,1.308500,0.840402
1000,0.839500,0.862964
1500,0.719200,1.040599
2000,0.452300,1.002799
2500,0.360600,1.349318


TrainOutput(global_step=2800, training_loss=0.6525462913513184, metrics={'train_runtime': 349.3341, 'train_samples_per_second': 64.122, 'train_steps_per_second': 8.015, 'total_flos': 2967322646937600.0, 'train_loss': 0.6525462913513184, 'epoch': 7.0})

In [13]:
from utils import compute_metrics

trainer_eval = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [14]:
trainer_eval.evaluate()

{'eval_loss': 1.2857354879379272,
 'eval_model_preparation_time': 0.0027,
 'eval_accuracy': {'accuracy': 0.6775},
 'eval_f1score': {'f1': 0.6747647004734894},
 'eval_runtime': 5.4078,
 'eval_samples_per_second': 147.935,
 'eval_steps_per_second': 18.492}

In [16]:
model.save_pretrained(model_name)
model.push_to_hub(f'nojedag/{model_name}')

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nojedag/distilroberta-finetuned-financial-news-sentiment-analysis_fr_gr/commit/9655c2655b0652cb344d332a0bd0cfee51f292d5', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='9655c2655b0652cb344d332a0bd0cfee51f292d5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nojedag/distilroberta-finetuned-financial-news-sentiment-analysis_fr_gr', endpoint='https://huggingface.co', repo_type='model', repo_id='nojedag/distilroberta-finetuned-financial-news-sentiment-analysis_fr_gr'), pr_revision=None, pr_num=None)

In [17]:
trainer.push_to_hub()
trainer_eval.push_to_hub()

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nojedag/distilroberta-finetuned-financial-news-sentiment-analysis_fr_gr/commit/6ea6e63e7865a2f9e9e7118bff60d764a291d5b5', commit_message='End of training', commit_description='', oid='6ea6e63e7865a2f9e9e7118bff60d764a291d5b5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nojedag/distilroberta-finetuned-financial-news-sentiment-analysis_fr_gr', endpoint='https://huggingface.co', repo_type='model', repo_id='nojedag/distilroberta-finetuned-financial-news-sentiment-analysis_fr_gr'), pr_revision=None, pr_num=None)